In [1]:
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.lm import MLE
import pandas as pd
import re
from ckiptagger import data_utils
from ckiptagger import WS
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.lm import KneserNeyInterpolated
from nltk.lm import Laplace
from nltk.lm import WittenBellInterpolated

## Pre-processing

* data_utils.download_data_url("/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data") 下載 CKIP 所需資料

* 引入 CKIP 的 word segmentation 資料

In [2]:

ws = WS("/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/data")


/Users/biaoyun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/biaoyun/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


* 讀入檔案（此份檔案為 yahoo 影評資料）

In [3]:

movie_review = pd.read_csv('/Users/biaoyun/Documents/111 Spring Semester Gtaduated Institute/Data/yahoo_movie_reviews.txt', sep='\t')
movie_review


,user_name,publish_time,comment,movies_key,movie_name,review_score
0,西瓜,2018-08-29 21:45:59,看完之後好難過😫也感到在浪費時間因為結局讓人好難過,0,復仇者聯盟：無限之戰,4.6
1,Innocent Eyes,2018-08-26 21:44:34,Marvel近期較差的片。大混戰處理得沒味道,0,復仇者聯盟：無限之戰,4.6
2,宏,2018-08-18 02:49:33,5顆星給在特效做得很棒，雖然有些地方還蠻瞎的,0,復仇者聯盟：無限之戰,4.6
3,品,2018-08-12 12:44:34,感覺不合邏輯!奇異博士為什麼要把時間寶石給他呀！一直把他困在時間迴圈裡不就得了嗎？結局 wh...,0,復仇者聯盟：無限之戰,4.6
4,隆,2018-08-12 00:57:40,有人說『對故事線不了解的無法體會情境』，我想我對於復仇者系列不是很瞭解，所以常常會出戲，感覺...,0,復仇者聯盟：無限之戰,4.6
...,...,...,...,...,...,...
7242,Chen,2018-02-28 14:17:03,我覺得普普通通，可以等待二輪片在看,49,犬之島,4.3
7243,柏樹,2018-02-28 14:05:32,真的很好看！結局出乎意料！女主角演技大耀進！,49,犬之島,4.3
7244,橘子,2018-02-28 13:06:57,個人覺得這部片的劇情很緊湊 給人緊張、刺激的興奮感但缺點是整體節奏有點快（根本是本人太慢速了...,49,犬之島,4.3
7245,阿豪,2018-02-28 12:11:23,真的超超超級好看超級大推 完全熱血 刺激 無冷場,49,犬之島,4.3


* 開始斷詞 （只取用 comment欄位）

In [4]:

token_list = []
for text in movie_review["comment"]:
  tokens = ws([text])
  token_list.append(tokens)
    

* 把斷完的 list 打開 （flatten）

In [5]:

flatten_tokens = []
for list_1 in token_list:
  for sublist in list_1:
    flatten_tokens.append(sublist)
    

## 模型訓練

### 沒有 smoothing 的原始模型

* 使用 3-grams 訓練模型

In [6]:

n = 3
train_pad_ngrams, flat_vocab = padded_everygram_pipeline(n, flatten_tokens)

model = MLE(n)    # initialize a model
model.fit(train_pad_ngrams, flat_vocab)

print(model.vocab)  # vocab used for training


<Vocabulary with cutoff=1 unk_label='<UNK>' and 15753 items>


* 印出 n-gram 數量

In [7]:

print(model.counts)


<NgramCounter with 3 ngram orders and 792381 ngrams>


* 去掉不需要的標示，讓生成的短文較好閱讀

In [8]:

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)


### 原始 3-grams 模型生成的短文

* 可以發現其實在還沒有 smoothing 之前所生成的內容就滿符合簡短的影評的特色。

In [9]:
generate_sent(model, 30, random_seed=100)

'劇情 唬爛 不 合 胃口 ， 也 不能 怪 他們 ， 主角 們 內心 層面 的 刻畫, 這 應該 是 低成本 的 國片.'

### 使用 KneserNeyInterpolated 訓練模型

In [10]:

n = 3
train_pad_ngrams, flat_vocab = padded_everygram_pipeline(n, flatten_tokens)

KNI_model = KneserNeyInterpolated(n)
KNI_model.fit(train_pad_ngrams, flat_vocab)

print(KNI_model.vocab)


<Vocabulary with cutoff=1 unk_label='<UNK>' and 15753 items>


### KneserNeyInterpolated 模型生成的短文

* 句子看起來較為複雜也仍算通順

In [11]:
generate_sent(KNI_model, 30, random_seed=100)

'人們 反思 一些 事情 ， 侷限 在 框框 中 ， 兩 個 半 小時 2200萬 的 成本 換到 極 大 收穫 的 一 部 喜劇   充滿 教育 意義 很'

## 以下因為好奇，所以多用幾種不同的 smoothing 方式訓練模型

### 使用 Laplace 訓練模型

In [12]:

n = 3
train_pad_ngrams, flat_vocab = padded_everygram_pipeline(n, flatten_tokens)

LAP_model = Laplace(n)
LAP_model.fit(train_pad_ngrams, flat_vocab)

print(LAP_model.vocab)


<Vocabulary with cutoff=1 unk_label='<UNK>' and 15753 items>


### Laplace 模型生成的短文

* 如果把「普通人」和「你」的位置調換結果就更好了呢 XD

In [13]:
generate_sent(LAP_model, 30, random_seed=100)

'前 二 集 加 在一起 算是 一 個 內容 不 夠 催淚, 大鬥 場面 又 不 多, 可能 是 因為 你 一定 會 受不了 ， 普通人'

### 使用 WittenBellInterpolated 訓練模型

In [14]:

n = 3
train_pad_ngrams, flat_vocab = padded_everygram_pipeline(n, flatten_tokens)

WBI_model = WittenBellInterpolated(n)
WBI_model.fit(train_pad_ngrams, flat_vocab)

print(WBI_model.vocab)


<Vocabulary with cutoff=1 unk_label='<UNK>' and 15753 items>


### WittenBellInterpolated 模型生成的短文

* 意外地竟然和原始沒有 smoothing 過的 3-grams 模型結果一樣！

In [15]:
generate_sent(WBI_model, 30, random_seed=100)

'劇情 唬爛 不 合 胃口 ， 也 不能 怪 他們 ， 主角 們 內心 層面 的 刻畫, 這 應該 是 低成本 的 國片.'